In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
import math

In [60]:
class RBM(nn.Module):

    def __init__(self, n_visible, n_hidden, CD_depth=5):
        super(RBM, self).__init__()
        # User input
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.CD_depth = CD_depth

        # Model Parameters
        initial_weight_variance = 1e-2
        self.W = nn.Parameter(torch.randn(n_visible, n_hidden)) * initial_weight_variance
        self.v_bias = nn.Parameter(torch.zeros(n_visible))
        self.h_bias = nn.Parameter(torch.zeros(n_hidden))

    def sample_from_p(self, p):
        """
        Sample a binary value from the probability distribution p. p is a 1d array with each value i corresponding
        to the probability that variable i is equal to 1. Naturally 1-p is the probablity of 0 then.

        Parameters:
            p - probability distribution

        Return:
            sample - the binary sample drawn from p
        """
        uniform_random = torch.rand(p.size())
        sample = F.relu(torch.sign(p  - uniform_random))
        return sample

    def v_to_h(self, v):
        h = F.linear(v, self.W.T, self.h_bias)
        p_h = F.sigmoid(h)
        h = self.sample_from_p(p_h)
        return h

    def h_to_v(self, h):
        v = F.linear(h, self.W, self.v_bias)
        p_v = F.sigmoid(v)
        v = self.sample_from_p(p_v)
        return v

    def forward(self, v):
        for _ in range(self.CD_depth):
            h = self.v_to_h(v)
            v = self.h_to_v(h)
        return v


In [67]:
# Initialize
epochs = 10
n_visible = 40
n_hidden = 20
lr = 0.01
CD_depth = 5

In [68]:
rbm = RBM(n_visible, n_hidden, CD_depth)

In [69]:
params = list(rbm.parameters())

for i in params:
    print(i.size())

torch.Size([40])
torch.Size([20])


In [74]:
# Init
dataset_size = 1000
batch_size = 100
n_batches = math.floor(dataset_size / batch_size)

# Read data
v_data = torch.randn(40, dataset_size).T

# Things to keep track of
loss_ = []

for _ in range(epochs):
    # Train network on batches of the data
    for i in range(n_batches):
        # Batch of input data
        v = v_data[i * batch_size: (i+1) * batch_size]

        # Pass data through network
        h = rbm.v_to_h(v)
        v_reconstructed = rbm(v)
        h_reconstructed = rbm.v_to_h(v_reconstructed)

        # Compute loss
        loss = lr * (v.mm(h.T).mean() - v_reconstructed.mm(v_reconstructed.T).mean())
        loss_.append(loss)
        
        # Update weights
        loss.backward()
        print(v.mm(h.T).mean().shape)

/home/karelgeraedts/Documents/uni/Master_Theses/code/Renormalization-Group-Bialek/.venv/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x40 and 20x100)

In [19]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Q = 3*a**3 - b**2

external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

print(a.grad, b.grad)

tensor([36., 81.]) tensor([-12.,  -8.])


In [ ]:
import numpy as np
from sklearn.neural_network import BernoulliRBM

def perform_nmcrg_analysis(activity_data, n_iterations):
    # Initialize an RBM model
    rbm = BernoulliRBM(n_components=activity_data.shape[1])

    # Fit the RBM model to the neural activity data
    rbm.fit(activity_data)

    # Perform the NMCRG procedure
    for _ in range(n_iterations):
        # Sample from the current RBM model
        samples = rbm.sample(activity_data.shape[0])

        # Fit a new RBM model to the samples
        rbm.fit(samples)

    return rbm

